# Store and retrieval


In [1]:
%pip install chromadb
%pip install langchain
%pip install langchain-community
%pip install langchain-chroma
%pip install langchain-huggingface
%pip install langchain-openai


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached langchain_huggingface-0.1.2-py3-none-any.whl.metadata (1.3 kB)
Using cached langchain_huggingface-0.1.2-py3-none-any.whl (21 kB)
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## 1. Load and split the documents

In [7]:
%load_ext dotenv
%dotenv

from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

# load the document and split it into chunks
loader = TextLoader("./content/max-und-moritz.txt")
documents = loader.load()

# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size = 2500, chunk_overlap = 200)
docs = text_splitter.split_documents(documents)

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


## 2. Store documents into the vector database

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

# prepare the embedding model
embedding_function = HuggingFaceEmbeddings(model_name = 'intfloat/multilingual-e5-large')

# load it into Chroma
db = Chroma(embedding_function = embedding_function, persist_directory = './chromadb/')
db.add_documents(docs)

/Users/christianweyer/Sources/gen-ai-customer-workshop-march-2025/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['3011e72e-b320-49cb-87b8-5de14de76e49',
 '7f83d56a-a109-4237-9e4f-82f862f79591',
 'd47d8e85-bbf7-4995-a70d-3098e0798f01',
 'bf789313-0614-4db1-aff1-42be975bd88b',
 '09932033-4679-4a24-870c-791836a0d043',
 '4b063420-48a7-4dfc-811c-65ef46fca906']

## 3. Query the documents

In [4]:
query = 'Wie heisst der Lehrer?'
#query = 'Who is the sensei?'
#query = 'Was machen die Hühner unmittelbar vor ihrem Tod?'
#query = 'Wie viele Hühner hatte Witwe Bolte?'
#query = 'Was machen Max und Moritz mit dem Schneider und wie heisst er?'

results = db.similarity_search_with_score(query)

for result in results:
    print(f'Score: {result[1]}')
    print(f'Text:\n{result[0].page_content}\n\n')

Score: 0.41745530977616524
Text:
– Nicht allein das A-B-C

Bringt den Menschen in die Höh’;

Nicht allein im Schreiben, Lesen

Übt sich ein vernünftig Wesen;

Nicht allein in Rechnungssachen

Soll der Mensch sich Mühe machen;

Sondern auch der Weisheit Lehren

Muß man mit Vergnügen hören. –

Daß dies mit Verstand geschah,

War Herr Lehrer Lämpel da. –

– Max und Moritz, diese beiden,

Mochten ihn darum nicht leiden;

Denn wer böse Streiche macht,

Gibt nicht auf den Lehrer acht. –

Nun war dieser brave Lehrer

Von dem Tobak ein Verehrer,

Was man ohne alle Frage

Nach des Tages Müh und Plage

Einem guten, alten Mann

Auch von Herzen gönnen kann. –

Max und Moritz, unverdrossen,

Sinnen aber schon auf Possen,

Ob vermittelst seiner Pfeifen

Dieser Mann nicht anzugreifen. –

– Einstens, als es Sonntag wieder

Und Herr Lämpel brav und bieder

In der Kirche mit Gefühle

Saß vor seinem Orgelspiele,

Schlichen sich die bösen Buben

In sein Haus und seine Stuben,

Wo die Meerschaumpfeife stan

## 4. RAG

In [8]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

retriever = db.as_retriever()

template = """Beantworte die Frage nur aufgrund der folgenenden Informationen:
{context}

Frage: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model_name = 'gpt-4o'
model = ChatOpenAI(model_name = model_name)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke(query)

'Der Lehrer heißt Herr Lehrer Lämpel.'